vamos a realizar un ***ETL*** de los data set de ***Steam*** proporcionados por la plataforma de bootcam de **Henry**.

En este caso los datos estan divididos en 3 dataset los cuales son **output_steam_games**, **australian_users_items** y **australian_users_reviews**.

En este notebook vamos a realizar el ***ETL*** del archivo **australian_users_items**

El archivo presenta porblemas de compatibilidad con **pandas** ya que no es un objeto JSON, su estructura es similar pero esta con comillas simples, por eso tambien usaremos el modulo **ast** para leer el arvhico como un diccionario de **Python**

In [1]:
### Importamos las librerias necesarias para la lectura 
import pandas as pd
import ast

file = './australian_users_items.json'

data = []
with open(file, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            ### Usamos ast.literal_eval para interpretar la línea como un diccionario de Python
            data_dict = ast.literal_eval(line)
            data.append(data_dict)
        except ValueError as e:
            print(f"Error al interpretar la línea: {e}")
            break  

# Convierte la lista de diccionarios a un DataFrame
df = pd.DataFrame(data)


C:\Users\Carlos Andres\AppData\Local\Temp\ipykernel_11740\1157169856.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Revisamos como queda nuentro DataFrame y visulizamos las primeras filas

In [2]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Creamos una funcion llamada ***extract_game_data***

Creamos una lista vacía llamada user_data para almacenar los datos desanidados de cada juego para el usuario.

Iteramos sobre cada juego en la lista de items utilizando un bucle for.

Para cada juego, extraemos información relevante, como el ID del juego, el nombre del juego, el tiempo de juego total y el tiempo de juego en las últimas 2 semanas, utilizando item.get('clave', valor_predeterminado) para asegurarnos de que obtengamos valores predeterminados si alguna clave no está presente en el diccionario del juego.

Creamos un diccionario llamado game_data que contiene esta información y lo agregamos a la lista user_data.

Finalmente, la función devuelve la lista user_data que contiene los datos desanidados de todos los juegos para el usuario.

In [3]:
### Función para extraer información de cada juego en la lista 'items'
def extract_game_data(items_list, user_id):
    
    user_data = []
    
    ### Iterar sobre cada juego en la lista de 'items'
    for item in items_list:
        ### Extraer información del juego
        game_data = {
            'user_id': user_id,
            'item_id': item.get('item_id', ''),
            'item_name': item.get('item_name', ''),
            'playtime_forever': item.get('playtime_forever', 0),
            'playtime_2weeks': item.get('playtime_2weeks', 0)
        }
        user_data.append(game_data)
    
    return user_data

### Lista para guardar los datos desanidados de todos los usuarios
all_data = []

### Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    ### Aplicar la función de desanidamiento
    user_games = extract_game_data(row['items'], row['user_id'])
    all_data.extend(user_games)

### Convertir la lista de diccionarios a DataFrame
df_desanidado = pd.DataFrame(all_data)

### Ahora 'df_desanidado' contiene la información desanidada
print(df_desanidado.head())

             user_id item_id                  item_name  playtime_forever  \
0  76561197970982479      10             Counter-Strike                 6   
1  76561197970982479      20      Team Fortress Classic                 0   
2  76561197970982479      30              Day of Defeat                 7   
3  76561197970982479      40         Deathmatch Classic                 0   
4  76561197970982479      50  Half-Life: Opposing Force                 0   

   playtime_2weeks  
0                0  
1                0  
2                0  
3                0  
4                0  


In [4]:
df_desanidado.info()
df_desanidado.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           object
 2   item_name         object
 3   playtime_forever  int64 
 4   playtime_2weeks   int64 
dtypes: int64(2), object(3)
memory usage: 196.6+ MB


,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0


Al ser porcas columnas las dejamos todas tal ves pueda hacer algo con ellas mas adelante

Convertimos el df a un archivo csv ya que me parece un mejor formato para manejar

In [6]:
cantidad_valores_con_datos = df_desanidado['item_name'].count()

# Imprime la cantidad de valores con datos en la columna 'mi_columna'
print(f'Cantidad de valores con datos en la columna "mi_columna": {cantidad_valores_con_datos}')

Cantidad de valores con datos en la columna "mi_columna": 5153209


In [46]:
df.to_csv('aust_users_items.csv', index=False)

In [47]:
df_desanidado.to_csv('aust_users_play_name.csv', index=False)